# Script to Update Hydraulic Grade Values and Interpolate Sample Points Using Ordinary Kriging

### Introduction

The Python component of this project was used to automate daily data imports and maintenance required to dependably produce leakage area results. Since the project is applied to a real-world water distribution system, data was generated daily, creating the need to automate labor intensive tasks. To do this, the pandas and geopandas python libraries were used to handle most of the data management by performing data imports, data cleanup, data table merges, and hydraulic grade calculations. This was accomplished by importing regularly generated pressure information into pandas data frames and hydrant locational information into a geopandas spatially enabled geodataframes. 

The static pressure update processing is an ongoing program at the water department, so an automated script was developed to capture additional hydrant pressure tests daily. This placed information indicating large areas exhibiting water leakage in front of decision-makers in a timely manner so that large water breaks can be identified and repaired. The script was written in Python and developed using Jupyter Notebooks in conjunction with ArcGIS to document each step and support replication in other water systems. 


In [ ]:
#Import the following python libraries
import sys, os, csv, fiona, datetime, arcpy
import pandas as pd
import numpy as np
import geopandas as gp
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt
from arcpy import env
from arcpy.sa import *

In [ ]:
#Set the environment workspace and overwrite settings
arcpy.env.overwriteOutput = True
arcpy.env.workspace = "C:\\StaticPressureProcess\\StaticPressureData.gdb"

In [ ]:
#Create variables for the pressure update csv, pressure test point file, and pressure zone polygon file
pressureUpdateFile = "C:\\StaticPressureProcess\\TasksExport.csv"
pressurePoint = "C:\\StaticPressureProcess\\StaticPressureData.gdb\\PZ1838A_PressureTestPnts"
pressureZone = "C:\\StaticPressureProcess\\StaticPressureData.gdb\\PZ1838A_Redefined"
outRaster = "C:\\StaticPressureProcess\\StaticPressureData.gdb\\LeakSurface_" + datetime.date.today().strftime("%m%d%Y")
geoStatModel = "C:\\StaticPressureProcess\\OrdinaryKrigingModel_1838A_TheBest.xml"
geoStatLayer = "KrigingOutLayer"

In [ ]:
#Use the fiona library to list all layers within the StaticPressureData geodatabase. 
#The list will be used to reference the layer imported with geopandas
fiona.listlayers("C:\\StaticPressureProcess\\StaticPressureData.gdb")

In [ ]:
#Import the PZ1838_PressureTestPnts feature class as a geodataframe.
#The layer parameter is taken from the fiona generated list position of the desired geodatabase feature class.
testSites = gp.read_file("C:\\StaticPressureProcess\\StaticPressureData.gdb",driver='FileGDB', layer=3)
testSites

In [ ]:
#Standardize the DateCollected column
testSites['DateCollected']=pd.to_datetime(testSites['DateCollected'])
testSites

In [ ]:
#Import the pressure updates csv into a pandas data frame
staticUpdates = gp.read_file( "C:\\StaticPressureProcess\\TasksExport.csv")
staticUpdates

In [ ]:
#Add the FACILITYID column and slice the text to only contain hydrant identifiers
staticUpdates['FACILITYID'] = staticUpdates.Asset.str[14:]

#Replace spaces with underscores
staticUpdates.columns = staticUpdates.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

#Convert the Static_Pressure column to numericvalues
staticUpdates['Static_Pressure']=pd.to_numeric(staticUpdates.Static_Pressure)
staticUpdates

In [ ]:
#Standardize the Actual_Stop_Date column
staticUpdates['Actual_Stop_Date']=pd.to_datetime(staticUpdates['Actual_Stop_Date'])
staticUpdates

In [ ]:
#Find and remove all rows with a Static_Pressure value equal to zero
zeroStaticP = staticUpdates[ staticUpdates['Static_Pressure'] == 0 ].index
staticUpdates.drop(zeroStaticP , inplace=True)

#Find and remove all rows with a Static_Pressure value greater than 200
zeroStaticP = staticUpdates[ staticUpdates['Static_Pressure'] > 300 ].index
staticUpdates.drop(zeroStaticP , inplace=True)

staticUpdates

In [ ]:
#Join the staticUpdates data frame to the testSites data frame using the FACILITYID field
#This creates a new data frame that contains the static pressure updates to apply to the 1838A test hydrants
mergedPressureInfo = testSites.merge(staticUpdates, on='FACILITYID')
mergedPressureInfo

In [ ]:
#Update new StaticPressure column
mergedPressureInfo.StaticPressure = mergedPressureInfo.Static_Pressure

#Recalculate the Hydrograde column
mergedPressureInfo.HydroGrade = mergedPressureInfo.Elevation + 2.31 * mergedPressureInfo.StaticPressure

#Update the DateCollected column with new dates from the Actual_Stop_Date column
mergedPressureInfo.DateCollected = mergedPressureInfo.Actual_Stop_Date
mergedPressureInfo

In [ ]:
#Remove unneeded fields from the data frame
del mergedPressureInfo['Task_ID']
del mergedPressureInfo['Asset']
del mergedPressureInfo['Activity']
del mergedPressureInfo['Static_Pressure']
del mergedPressureInfo['Actual_Stop_Date']
del mergedPressureInfo['geometry_y']

#Rename the geometry column
mergedPressureInfo.rename(columns={"geometry_x":"geometry"}, inplace=True)
mergedPressureInfo

In [ ]:
#Remove duplicate values
mergedPressureInfo = mergedPressureInfo.sort_values('DateCollected',ascending=True)
mergedPressureInfo = mergedPressureInfo.drop_duplicates(subset='FACILITYID', keep='first')
mergedPressureInfo = mergedPressureInfo.sort_values('FACILITYID',ascending=True)
mergedPressureInfo

## Update testSite values with the new static pressure test values and export to a shapefile

In [ ]:
#Set the testSites index to the FACILITYID column
testSites = testSites.set_index('FACILITYID')

In [ ]:
testSites

In [ ]:
#Set the mergedPressureInfo data frame index to the FACILITYID column
mergedPressureInfo = mergedPressureInfo.set_index('FACILITYID')

In [ ]:
mergedPressureInfo

In [ ]:
#Run the update function on the testSites data frame
testSites.update(mergedPressureInfo)

In [ ]:
#Reset the indexes
testSites.reset_index(inplace=True)

In [ ]:
testSites

In [ ]:
#Convert the merged data frame to a GeoDataFrame and remove null HydroGrade Values
updatedGdf = gp.GeoDataFrame(testSites, geometry='geometry')
NewGdf = updatedGdf[updatedGdf.HydroGrade.notnull()]
NewGdf

In [ ]:
#Convert the DateCollected column to string values in order to export to shapefile
NewGdf['DateCollected']=NewGdf['DateCollected'].astype(str)

In [ ]:
#Set the new geodataframe's projection and plot the new pressure tests within 1838A
NewGdf.crs = {"init":"epsg:2274"}
updatedGdf.plot(figsize=(12,12));

In [ ]:
##Create new shapefile name and export the geodataframe to new shapefile
shpFileName = r"C:\StaticPressureProcess\UpdatedStaticPressureTests_" + datetime.date.today().strftime("%m%d%Y") + ".shp"

In [ ]:
NewGdf.to_file(shpFileName)

## Run the ordinary kriging model on the updated hydrant pressure points

Run the Kriging interpolation using the pressure point layer. This step creates a Geostatistical Layer using tools from Geostatistical Analyst. The tool uses an existing Geostatistial layer as a model source to duplicate its parameters and should be stored in the project workspace.

In [ ]:
#Check out the ESRI Spatial and Geostatistical Analyst Extensions
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("GeoStats")

In [ ]:
krigingInLayer = "C:\\StaticPressureProcess\\UpdatedStaticPressureTests_07192019.shp X=Shape Y=Shape F1=HydroGrade"

In [ ]:
#arcpy.GACreateGeostatisticalLayer_ga(in_ga_model_source, in_datasets, out_layer)
arcpy.GACreateGeostatisticalLayer_ga(geoStatModel, krigingInLayer, geoStatLayer)

In [ ]:
#Create the pressure surface based on pressurePoints
arcpy.Kriging_3d(shpFileName, "HydroGrade", outRaster)

In [ ]:
#Clip the interpolation surface to the desired polygon boundary layer
arcpy.Clip_analysis(outRaster, pressureZone)

In [ ]:
#Check back in the ESRI Spatial and Geostatistical Analyst Extensions
arcpy.CheckInExtension("Spatial")
arcpy.CheckInExtension("GeoStats")

In [ ]:
print("Completed Script")